In [1]:
import json
import xml.etree.ElementTree as ET

def getName(nameList):
    for name in nameList:
        if 'language' in name and name['language'] == 'en':
           return name
    return nameList[0]
    

In [2]:

def opendoarMap(line):
    entry = ET.Element("registryEntry")
    repo = json.loads(line)
    ET.SubElement(entry, "identifier", type = "URI").text = repo['system_metadata']['uri']
    if repo['system_metadata']['id'] != '':
        ET.SubElement(entry, 'internalIdentifier').text = str(repo['system_metadata']['id'])
    dic = repo['repository_metadata']
    if 'name' in dic and dic['name'] != []:
        name = getName(dic["name"])
        if 'language' in name:
            ET.SubElement(entry, "name", nameLanguage = name['language'], languageVocabulary = "ISO 639-1").text = name['name']
        else:
            ET.SubElement(entry, "name").text = name['name']
        if 'acronym' in name:
            ET.SubElement(entry, "additionalName").text = name['acronym']
    if 'url' in dic and dic['url'] != '':
        ET.SubElement(entry,'url').text = dic['url']
    if 'type' in dic and dic['type'] != '':
        ET.SubElement(entry,'type').text = dic['type']
    if 'content_types' in dic and dic['content_types'] != []:
        for content in dic['content_types']:
            ET.SubElement(entry,'content').text = content
    if 'metadata_record_count' in dic and dic['metadata_record_count'] != '':
        ET.SubElement(entry, "recordCount").text = str(dic['metadata_record_count'])
    if 'content_subjects' in dic :
        for sbj in dic['content_subjects']:
            ET.SubElement(entry,'subject').text = sbj
    org = repo['organisation']
    if 'name' in org and org['name'] != []:
        name = getName(org['name'])
        organization = ET.Element("organization")
        if 'language' in name:
            ET.SubElement(organization,"name", nameLanguage = name['language'], languageVocabulary = "ISO 639-1").text = name['name']
        else:
            ET.SubElement(organization,"name").text = name['name']
        if 'acronym' in name:
            ET.SubElement(organization,"acronym").text = name['acronym']
        if 'country' in org and org['country'] != '':
            ET.SubElement(organization,"country").text = org['country']
        if 'url' in org and org['url'] != '':
            ET.SubElement(organization,"organizationUrl").text = org['url']
        if 'identifiers' in org and org['identifiers'] != []:
            for ide in org['identifiers']:
                if 'type' in ide:
                    ET.SubElement(organization,"id", type = ide['type']).text = ide['identifier'] 
                else: 
                    ET.SubElement(organization,"id").text = ide['identifier']
        entry.append(organization)
    if 'software' in dic and 'name' in dic['software'] and dic['software']['name'] != '':
        ET.SubElement(entry,'softwareName').text = dic['software']['name']
    if 'oai_url' in dic and dic['oai_url'] != '':
        ET.SubElement(entry,'apiUrl', type="OAI-PMH").text = dic['oai_url']
    ET.SubElement(entry, "access", type="open").text = "dataAccess"
    ET.SubElement(entry, "access", type="open").text = "dataDeposition"
    if 'policy_urls' in dic:
        type = 0
        url = 0
        for p in dic['policy_urls']:
            if 'policy_url' in p and p['policy_url'] != '':
                ET.SubElement(entry, "policyUrl").text = p['policy_url']
    if repo['system_metadata']['date_created'] != '':
        ET.SubElement(entry, 'startDate', format = "yyyy-MM-dd").text = repo['system_metadata']['date_created'].split(" ")[0]
    if repo['system_metadata']['date_modified'] != '':
        ET.SubElement(entry, 'updateDate', format = "yyyy-MM-dd").text = repo['system_metadata']['date_modified'].split(" ")[0]
    return entry
    
    

In [3]:
od = open('../data/opendoar_dump_2023-02-01.json')

entry = ET.Element("registryEntries")

for line in od:
    if line == "":
        continue
    entry.append(opendoarMap(line))


tree = ET.ElementTree(entry)
tree.write("../commonModel/od.xml")

